In [1]:
import sys
sys.path.append('../sources/')
import pandas as pd
#from geotools import distance2d, calculate_bearing_angle, dfPoint2LonLat
import numpy as np
import sumolib
import network
from cleandata import richdata
from geotools import dfPoint2LonLat

In [2]:
net1 = sumolib.net.readNet("../../sumo_iland_map/iland.net" ,withInternal=False)
mynet = network.Net()
mynet.importFromSumoNet(net1)

gps = pd.read_csv("../../GTFS/KoDa/gps_inside_island.csv",dtype={"trip_id":object,
                                                              "shape_id":object,
                                                              "route_id":object,
                                                             "vehicle_id":object,
                                                             "timestamp":int,
                                                             "direction_id":int,
                                                             "lat":float,
                                                             "lon":float,
                                                             "speed":float,
                                                             "bearing":int,
                                                             "inside":bool,
                                                             "trip_id _extended":object})

unique_trip = list(gps["trip_id_extended"].unique())

In [3]:
sample_gps = gps[gps["trip_id_extended"]==unique_trip[2530]].sort_values("timestamp").drop_duplicates()
sample_gps = sample_gps.reset_index(drop=True)
sample_gps

,vehicle_id,timestamp,trip_id,route_id,direction_id,lat,lon,speed,bearing,shape_id,inside,trip_id_extended
0,9031001001004012,1685966492,14010000623439929,9011001000400000,0,59.303558,18.079140,11.4,342,1014010000623425442,True,14010000623439929_0
1,9031001001004012,1685966494,14010000623439929,9011001000400000,0,59.303757,18.079018,11.4,342,1014010000623425442,True,14010000623439929_0
2,9031001001004012,1685966496,14010000623439929,9011001000400000,0,59.303951,18.078894,11.7,342,1014010000623425442,True,14010000623439929_0
3,9031001001004012,1685966498,14010000623439929,9011001000400000,0,59.304150,18.078768,11.7,342,1014010000623425442,True,14010000623439929_0
4,9031001001004012,1685966500,14010000623439929,9011001000400000,0,59.304352,18.078644,11.7,342,1014010000623425442,True,14010000623439929_0
...,...,...,...,...,...,...,...,...,...,...,...,...
474,9031001001004012,1685967508,14010000623439929,9011001000400000,0,59.318768,18.032166,7.8,351,1014010000623425442,True,14010000623439929_0
475,9031001001004012,1685967511,14010000623439929,9011001000400000,0,59.318985,18.032099,8.6,351,1014010000623425442,True,14010000623439929_0
476,9031001001004012,1685967514,14010000623439929,9011001000400000,0,59.319141,18.032032,9.2,347,1014010000623425442,True,14010000623439929_0
477,9031001001004012,1685967516,14010000623439929,9011001000400000,0,59.319397,18.031898,10.3,344,1014010000623425442,True,14010000623439929_0


In [4]:
df  = richdata(sample_gps,mynet, alpha=.7)
df = dfPoint2LonLat(df, mynet)
df.head()

,id,timestamp,speed,bearing,stopindex,lon,lat
0,0,1685966492,11.582420,345.043530,0,18.079140,59.303558
1,0,1685966494,11.439408,344.598304,0,18.079018,59.303757
2,0,1685966496,11.523337,344.416486,0,18.078894,59.303952
3,0,1685966498,11.781406,344.922684,0,18.078768,59.304149
4,0,1685966500,11.554198,344.954263,0,18.078643,59.304350


In [5]:
df1 = df.merge(sample_gps, on="timestamp")
df1 = df1.rename(columns={"lon_y":"lon", "lat_y":"lat"})
df1 .to_csv("test.csv")

In [6]:
df1.iloc[0]

id                                    0
timestamp                    1685966492
speed_x                        11.58242
bearing_x                     345.04353
stopindex                             0
lon_x                          18.07914
lat_x                         59.303558
vehicle_id             9031001001004012
trip_id               14010000623439929
route_id               9011001000400000
direction_id                          0
lat                           59.303558
lon                            18.07914
speed_y                            11.4
bearing_y                           342
shape_id            1014010000623425442
inside                             True
trip_id_extended    14010000623439929_0
x                           2999.822757
y                            248.409699
Name: 0, dtype: object

## The steps of the data riching and cleaning:
#### 1 - convert to x, y
#### 2 - remove outlier
#### 3 - smoothing vx and vy and update x,y
#### 4 - calculate bearing and speed
#### 5 - assing zero to speeds less than MINSPEED_FOR_MOVING
#### 6 - update bearings based on speeds less than MINSPEED_FOR_BEARING
#### 7 - assing stopindex and fix locations in stop cases


In [7]:
"""
MAX_SPEED_FOR_OUTLIER = 50

df1 = df.join(df.shift(-1).rename(columns={"timestamp":"timestamp2","x":"x2","y":"y2"}))
df1.at[df1.index[-1],"timestamp2"] = df1.iloc[-1]["timestamp"] + 1
df1.at[df1.index[-1],"x2"] = df1.iloc[-1]["x"]
df1.at[df1.index[-1],"y2"] = df1.iloc[-1]["y"]
df1["dist"] = df1.apply(lambda row: distance2d((row["x"],row["y"]),(row["x2"],row["y2"])), axis=1)
df1["dtime"] = df1.apply(lambda row: int(row["timestamp2"] - row["timestamp"]), axis=1)
df1["speed"] = df1.apply(lambda row: row["dist"]/row["dtime"], axis=1)
df2 = df1[df1["speed"]<MAX_SPEED_FOR_OUTLIER].reset_index(drop=True)
"""

'\nMAX_SPEED_FOR_OUTLIER = 50\n\ndf1 = df.join(df.shift(-1).rename(columns={"timestamp":"timestamp2","x":"x2","y":"y2"}))\ndf1.at[df1.index[-1],"timestamp2"] = df1.iloc[-1]["timestamp"] + 1\ndf1.at[df1.index[-1],"x2"] = df1.iloc[-1]["x"]\ndf1.at[df1.index[-1],"y2"] = df1.iloc[-1]["y"]\ndf1["dist"] = df1.apply(lambda row: distance2d((row["x"],row["y"]),(row["x2"],row["y2"])), axis=1)\ndf1["dtime"] = df1.apply(lambda row: int(row["timestamp2"] - row["timestamp"]), axis=1)\ndf1["speed"] = df1.apply(lambda row: row["dist"]/row["dtime"], axis=1)\ndf2 = df1[df1["speed"]<MAX_SPEED_FOR_OUTLIER].reset_index(drop=True)\n'

In [131]:
import numpy as np
from scipy.stats import gaussian_kde

def kernel_density_smoothing(data, bandwidth):
    kde = gaussian_kde(data, bw_method=bandwidth)
    smoothed_data = kde(data)
    return smoothed_data

# Example usage
original_data = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
bandwidth = 3.0
smoothed_data = kernel_density_smoothing(original_data, bandwidth)
print(smoothed_data)

[0.00415553 0.00435524 0.00450371 0.0045952  0.00462611 0.0045952
 0.00450371 0.00435524 0.00415553]


In [136]:
import numpy as np
from scipy.ndimage import convolve1d

def convolution_smoothing(data, window_size):
    window = np.ones(window_size) / window_size
    smoothed_data = convolve1d(data, window, mode='nearest')
    return smoothed_data

# Example usage
original_data = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
window_size = 3
smoothed_data = convolution_smoothing(original_data, window_size)
print(smoothed_data)

[13 20 30 40 49 60 70 80 86]
